In [9]:
import torch
from model import GPT2, GPT2Config
import jax
import jax.numpy as jnp

In [2]:
sd = torch.load('ckpt.pt')

In [3]:
model = sd['model']
sd['step'], sd['best_val_loss']

(3500, Array(1.916147, dtype=float32))

In [37]:
#sd.keys() # save config in future
config = GPT2Config(
    seq_len     = 64,
    num_layers  = 4,
    num_heads   = 4,
    hidden_dims = 128,
    vocab_size  = 65,
    dropout     = 0.0
)
raw_model = GPT2.init(config, key=jax.random.PRNGKey(0))

In [38]:
loaded_model = raw_model.from_state_dict(model, prefix=None)

In [55]:

out = loaded_model.generate(
        seq=jnp.array([0]),
        max_new_tokens=50,
        temperature=1.0,
        key=jax.random.PRNGKey(0)
)

In [53]:
out[-51:].tolist()

[0, 32, 56, 43, 39, 58, 1, 58, 47, 57, 43]

In [45]:
import os
import pickle

# Assuming 'meta.pkl' is a pickle file
file_path = os.path.join('data/shakespeare', 'meta.pkl')

with open(file_path, 'rb') as file:
    # Load the content of the file
    data = pickle.load(file)

# Now you can use the 'data' variable which contains the data from 'meta.pkl'


In [49]:
itos = data['itos']
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [56]:
decode(out[-51:].tolist())

'\nTreat tisent degenson. Naward and his your me!\nI t'